In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import re

In [2]:
def news_collection():
        urls = ["https://www.yonhapnewstv.co.kr/browse/feed/",                  #최신 1
                "http://www.yonhapnewstv.co.kr/category/news/politics/feed/",   # 정치 2
                "http://www.yonhapnewstv.co.kr/category/news/economy/feed/",    # 경제 3
                "http://www.yonhapnewstv.co.kr/category/news/society/feed/",    # 사회 4
                
                "https://news.sbs.co.kr/news/ReplayRssFeed.do?prog_cd=R1&plink=RSSREADER",     # 정치 5
                "https://news.sbs.co.kr/news/SectionRssFeed.do?sectionId=02&plink=RSSREADER",  # 경제 6
                "https://news.sbs.co.kr/news/SectionRssFeed.do?sectionId=03&plink=RSSREADER",  # 사회 7
                
                "https://www.newsis.com/RSS/sokbo.xml",    # 속보 8 
                "https://www.newsis.com/RSS/politics.xml", # 정치 9
                "https://www.newsis.com/RSS/economy.xml",  # 경제 10
                "https://www.newsis.com/RSS/society.xml",  # 사회 11
                ]

        news = ["연합뉴스", #1
                "연합뉴스", #2
                "연합뉴스", #3
                "연합뉴스", #4
                
                "SBS", #5
                "SBS", #6
                "SBS", #7

                "뉴시스", #8
                "뉴시스", #9
                "뉴시스", #10
                "뉴시스", #11
                ]

        categories = ["최신",  #1
                "정치",  #2
                "경제",  #3
                "사회",  #4
                
                "정치",  #5
                "경제",  #6
                "사회",  #7
                
                "속보",  #8
                "정치",  #9
                "경제",  #10
                "사회",  #11
                ]
        
        # 기본 빈 DataFrame 생성
        df1 = pd.DataFrame( columns=["Media", "Category", "Title", "Link", "Description", "Publication Date"] )

        def remove_between_chars(input_str, char1, char2):
                pattern = re.compile(re.escape(char1) + '.*?' + re.escape(char2))
                return re.sub(pattern, '', input_str)

        def news_colltect(df1,urls, news, categories):
        # urls 목록을 기반으로 데이터를 반복하여 가져옴
                for i in range(len(urls)):
                        
                        # Retrieve the XML data
                        response = requests.get(urls[i])
                        xml_data = response.content
                        # Parse the XML data using BeautifulSoup
                        soup = BeautifulSoup(xml_data, "xml")
                        # Extracting items
                        items = soup.find_all('item')
                        # Parsing each item
                        data = []
                        
                        for item in items:
                                title = item.find('title').text
                                link = item.find('link').text
                                pubDate = item.find('pubDate').text
                                media = news[i]
                                # description = item.find('description').text
                                if item.find('content:encoded'):
                                        description = item.find('content:encoded').text
                                else : 
                                        description = item.find('description').text
                                
                                catego = categories[i]
                                data.append({"Media": media, "Category" : catego, "Title": title, "Link": link, "Description": description, "Publication Date": pubDate})
                                
                        # Progressing 행현황 점검
                        print(f"{i+1}. {news[i]}_{catego}, {urls[i]}")

                        # Convert to pandas DataFrame    
                        df = pd.DataFrame(data)
                        df['Publication Date'] = pd.to_datetime(df['Publication Date']).apply(lambda x: x.date().strftime('%Y-%m-%d'))
                        
                        # df.to_csv(f"{news[i]}.csv", encoding="utf8", index=False)
                        df1 = pd.concat( [df1, df], axis=0).reset_index(drop=True)
        
        
                # concat df파일 불필요한 부분 제거
                
                df1['Description'] = df1['Description'].apply(lambda x: remove_between_chars(x, "<a","/a>") )
                df1['Description'] = df1['Description'].apply(lambda x: remove_between_chars(x, "<!-- tracking Pixel -->","<!-- //tracking Pixel -->"))
                df1['Description'] = df1['Description'].apply(lambda x: remove_between_chars(x, "<iframe","</iframe>"))
                df1['Description'] = df1['Description'].apply(lambda x: remove_between_chars(x, "<",">"))
                df1['Description'] = df1['Description'].apply(lambda x: remove_between_chars(x, "&#",";"))


                df1['Description'] = df1['Description'].apply(lambda x: x.replace("\n"," "))
                df1['Description'] = df1['Description'].apply(lambda x: x.replace("\r"," "))
                df1['Description'] = df1['Description'].apply(lambda x: x.replace("\t",""))
                df1['Description'] = df1['Description'].apply(lambda x: x.replace("&quot;"," "))
                df1['Description'] = df1['Description'].apply(lambda x: x.replace("&nbsp;"," "))
                df1['Description'] = df1['Description'].apply(lambda x: x.replace("    ",""))
                df1['Description'] = df1['Description'].apply(lambda x: x.replace("   ",""))
                df1['Description'] = df1['Description'].apply(lambda x: x.replace("  ",""))
                df1['Description'] = df1['Description'].apply(lambda x: x.replace("ⓒ SBS &amp; SBS Digital News Lab: 무단복제 및 재배포 금지",""))
                df1['Description'] = df1['Description'].apply(lambda x: x.replace("<p class='change'>",""))
                
                df1['Title'] = df1['Title'].apply(lambda x: x.replace("&quot;"," "))
                df1['Title'] = df1['Title'].apply(lambda x: remove_between_chars(x, "&#",";"))
                        
                now = datetime.now()
                today = f"{now.date()}_{now.hour:0>2}_{now.minute:0>2}"
                
                df1.to_csv(f"news_{today}.csv", encoding="utf8", index=False)
                df1.to_excel (f"news_{today}.xlsx", index=False)
                return df1
        
        df_news = news_colltect(df1, urls, news, categories)
        return df_news
                

In [4]:
%%time
news_collection()
# CPU times: total: 4.16 s
# Wall time: 5.02 s

1. 연합뉴스_최신, https://www.yonhapnewstv.co.kr/browse/feed/
2. 연합뉴스_정치, http://www.yonhapnewstv.co.kr/category/news/politics/feed/
3. 연합뉴스_경제, http://www.yonhapnewstv.co.kr/category/news/economy/feed/
4. 연합뉴스_사회, http://www.yonhapnewstv.co.kr/category/news/society/feed/
5. SBS_정치, https://news.sbs.co.kr/news/ReplayRssFeed.do?prog_cd=R1&plink=RSSREADER
6. SBS_경제, https://news.sbs.co.kr/news/SectionRssFeed.do?sectionId=02&plink=RSSREADER
7. SBS_사회, https://news.sbs.co.kr/news/SectionRssFeed.do?sectionId=03&plink=RSSREADER
8. 뉴시스_속보, https://www.newsis.com/RSS/sokbo.xml
9. 뉴시스_정치, https://www.newsis.com/RSS/politics.xml
10. 뉴시스_경제, https://www.newsis.com/RSS/economy.xml
11. 뉴시스_사회, https://www.newsis.com/RSS/society.xml
CPU times: total: 4.16 s
Wall time: 5.02 s


,Media,Category,Title,Link,Description,Publication Date
0,연합뉴스,최신,"피치, 한국 신용등급 'AA-' 유지…등급전망 '안정적'",https://www.yonhapnewstv.co.kr/news/MYH2024030...,"피치, 한국 신용등급 'AA-' 유지…등급전망 '안정적'국제 신용평가사 피치가 한국...",2024-03-06
1,연합뉴스,최신,"테러범 진압 ""쾅!""…국가중요시설 테러대응 훈련",https://www.yonhapnewstv.co.kr/news/MYH2024030...,"테러범 진압 ""쾅!""…국가중요시설 테러대응 훈련[앵커]서울 여의도 한복판에 테러범이...",2024-03-06
2,연합뉴스,최신,"30초만에 금은방 털고 도주…경찰, 2인조 절도범 추적",https://www.yonhapnewstv.co.kr/news/MYH2024030...,"30초만에 금은방 털고 도주…경찰, 2인조 절도범 추적경찰이 새벽 시간 금은방 문을...",2024-03-06
3,연합뉴스,최신,계속된 수업거부에 개강일 안갯속…수업 가능 마지노선은?,https://www.yonhapnewstv.co.kr/news/MYH2024030...,계속된 수업거부에 개강일 안갯속…수업 가능 마지노선은?[앵커]의대 정원 확대에 반발...,2024-03-06
4,연합뉴스,최신,법원으로 간 '의대 증원'…행정소송 쟁점은,https://www.yonhapnewstv.co.kr/news/MYH2024030...,법원으로 간 '의대 증원'…행정소송 쟁점은[앵커]의대 증원 문제를 놓고 정부와 의사...,2024-03-06
...,...,...,...,...,...,...
306,뉴시스,사회,전공의 이탈에…처방전 없이도 건보 환급 가능해진다,http://www.newsis.com/view/?id=NISX20240306_00...,[서울=뉴시스] 구무서 기자 = 전공의 이탈 장기화로 병원 진료에 차질이 빚어지자 ...,2024-03-06
307,뉴시스,사회,천범산 충북교육청 부교육감 봉명초 찾아 늘봄학교 점검,http://www.newsis.com/view/?id=NISX20240306_00...,[청주=뉴시스] 김재광 기자 = 천범산 충북도교육청 부교육감이 6일 새 학기부터 운...,2024-03-06
308,뉴시스,사회,"심신미약 상태서 삼촌 살인미수 혐의 50대, 징역형",http://www.newsis.com/view/?id=NISX20240306_00...,[대구=뉴시스] 김정화 기자 = 망상장애 등 심신미약 상태에서 삼촌을 살해하려다 미...,2024-03-06
309,뉴시스,사회,새벽시간대 30초 만에 금은방 턴 2인조…경찰 추적,http://www.newsis.com/view/?id=NISX20240306_00...,[인천=뉴시스] 김동영 기자 = 새벽시간대 신원을 알 수 없는 2명이 금은방 유리문...,2024-03-06
